# Disruption of the Sagattarius Dwarf Galaxy in WWT

<img src="https://raw.githubusercontent.com/marksubbarao/pyWWT_AAS225/master/images/SgrStream.png">

We will visualize an N-body simulation of the tidal disruption of the Sagattarius Dwarf galaxy as described in Law and Majewski 2012 http://iopscience.iop.org/0004-637X/714/1/229/article Data files were taken from the website: http://www.astro.virginia.edu/~srm4n/Sgr/data.html 

In [1]:
#Set up astropy and astroquery 
from astropy.table import Table,Column
from astropy.coordinates import SkyCoord, Distance, CartesianRepresentation
from astropy import units as u

C:\Anaconda\lib\site-packages\IPython\kernel\__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


## The Star Streams

In [2]:
#Import data set, keep only the columns we need. Convert distances from kpc to pc for WWT
Stars = Table.read('data/SgrTriax_DYN.dat', format='ascii.basic')
Stars.keep_columns(["ra","dec","dist","Pcol"])
Stars['ra'].unit = u.degree
Stars['dec'].unit = u.degree
distCol=Column(1000*Stars['dist'],name='distance',meta={'ucd': 'pos.distance'}, unit='pc')
Stars.add_column(distCol)
del Stars['dist']
Stars

ra,dec,Pcol,distance
deg,deg,,pc
float64,float64,int32,float64
295.968,-42.1607,8,65975.1
284.571,-31.0457,-1,28705.5
300.004,-31.3875,0,24114.8
279.433,-35.746,8,62416.9
267.578,-38.2792,7,49910.6
176.893,1.10254,4,47122.8
104.444,33.1628,4,53437.1
71.5963,-0.34414,5,25899.1


#### Add colormap column based on Pcol as described in LM10. Black changed to grey for visibility

In [3]:
Stars.remove_columns(["colormap"])

KeyError: u'Column colormap does not exist'

In [4]:
Stars['colormap'] = 'magenta'
for i in range(len(Stars['Pcol'])):
  pc=Stars['Pcol'][i]
  if pc==-1:
    Stars['colormap'][i]='grey'
  elif pc==0 or pc==1:
    Stars['colormap'][i]='orange'
  elif pc==2 or pc==3:
    Stars['colormap'][i]='magenta'
  elif pc==4 or pc==5:
    Stars['colormap'][i]='cyan'
  elif pc==6 or pc==7:
    Stars['colormap'][i]='green'
Stars

ra,dec,Pcol,distance,colormap
deg,deg,,pc,
float64,float64,int32,float64,string56
295.968,-42.1607,8,65975.1,magenta
284.571,-31.0457,-1,28705.5,grey
300.004,-31.3875,0,24114.8,orange
279.433,-35.746,8,62416.9,magenta
267.578,-38.2792,7,49910.6,green
176.893,1.10254,4,47122.8,cyan
104.444,33.1628,4,53437.1,cyan
71.5963,-0.34414,5,25899.1,cyan


### Visualize Star Streams in WWT

In [5]:
#Connect to WWT and set up Layer Group
from pywwt.mods import *
wwt = WWTClient(host="127.0.0.1") #Can pass a IP address here if WWT is running on a remote machine
wwt.new_layer_group("Sky","Sgr Dwarf Disruption")

Connecting to WWT on host 127.0.0.1.


In [6]:
#Set up WWT layer
SgrStream_layer = wwt.new_layer("Sgr Dwarf Disruption", "SgrStream", Stars.colnames)
#Set visualization parameters in WWT
props_dict = {"CoordinatesType":"Spherical",\
              "MarkerScale":"Screen",\
              "PointScaleType":"Constant",\
              "ScaleFactor":"8",\
              "ShowFarSide":"True",\
              "AltColumn":"3",\
              "AltType":"Distance",\
              "AltUnit":"Parsecs",\
              "RaUnits":"Degrees",\
              "PlotType":"Gaussian",\
              "ColorMapColumn":"4",\
              "TimeSeries":"False"}
SgrStream_layer.set_properties(props_dict)
#Send data to WWT client
SgrStream_layer.update(data=Stars, purge_all=True, no_purge=False, show=True)

## Sgr Dwarf Orbit

In [6]:
Orbit = Table.read('data/SgrTriax_orbit.dat', format='ascii.no_header')
Orbit.keep_columns(["col1","col4","col5","col12"])
Orbit.rename_column('col1', 'time')
Orbit.rename_column('col4', 'RA')
Orbit.rename_column('col5', 'dec')
Orbit.rename_column('col12', 'dist')
distCol=Column(1000*Orbit['dist'],name='distance',meta={'ucd': 'pos.distance'}, unit=u.pc)
Orbit.add_column(distCol)
del Orbit['dist']
Orbit['RA'].unit = u.degree
Orbit['dec'].unit = u.degree
Orbit['time'].unit = u.Gyr

In [12]:
Orbit

time,RA,dec,distance
Gyr,deg,deg,pc
-7.9961,29.7451,-8.51732,52743.6
-7.9889,28.869,-8.92598,53198.1
-7.9817,27.9937,-9.33262,53632.8
-7.9744,27.1194,-9.73709,54049.6
-7.9672,26.2453,-10.1392,54446.9
-7.96,25.37,-10.5403,54826.8
-7.9527,24.4935,-10.9395,55188.0
-7.9455,23.6163,-11.3363,55531.5
-7.9383,22.7355,-11.7326,55857.7


### Setting up the Time Series

WWT can't advance time over billions of years. Here we set up a column in the WWT time format scaled so that one day equals one billion years in the simulation.

In [17]:
from astropy.time import Time, TimeDelta
T0 = Time('2014-01-01T00:00:00.0', format='isot', scale='utc')
TimeCol=T0+TimeDelta(Orbit['time'],format='jd')
wwtCol=[]
for timeElement in TimeCol:
  wwtCol.append(timeElement.datetime.strftime('%m/%d/%Y %H:%M:%S'))

Orbit.add_column(Column(wwtCol,name="proxyTime"))

In [18]:
Orbit

time,RA,dec,distance,proxyTime
Gyr,deg,deg,pc,
-7.9961,29.7451,-8.51732,52743.6,12/24/2013 00:05:36
-7.9889,28.869,-8.92598,53198.1,12/24/2013 00:15:59
-7.9817,27.9937,-9.33262,53632.8,12/24/2013 00:26:21
-7.9744,27.1194,-9.73709,54049.6,12/24/2013 00:36:51
-7.9672,26.2453,-10.1392,54446.9,12/24/2013 00:47:13
-7.96,25.37,-10.5403,54826.8,12/24/2013 00:57:36
-7.9527,24.4935,-10.9395,55188.0,12/24/2013 01:08:06
-7.9455,23.6163,-11.3363,55531.5,12/24/2013 01:18:28
-7.9383,22.7355,-11.7326,55857.7,12/24/2013 01:28:50


In [19]:
#Set up WWT layer
SgrOrbit_layer = wwt.new_layer("Sgr Dwarf Disruption", "SgrOrbit", Orbit.colnames)
#Set visualization parameters in WWT
props_dict = {"CoordinatesType":"Spherical",\
              "MarkerScale":"Screen",\
              "PointScaleType":"Constant",\
              "ScaleFactor":"8",\
              "ShowFarSide":"True",\
              "AltColumn":"3",\
              "AltType":"Distance",\
              "AltUnit":"Parsecs",\
              "RaUnits":"Degrees",\
              "PlotType":"Gaussian",\
              "TimeSeries":"False"}
SgrOrbit_layer.set_properties(props_dict)
#Send data to WWT client
SgrOrbit_layer.update(data=Orbit, purge_all=True, no_purge=False, show=True)